In [18]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [2]:
airbnbDataSet_filename = os.path.join(os.getcwd(), "airbnbListingsData.csv")

df = pd.read_csv(airbnbDataSet_filename, header=0)

df.head()

len(df.columns)
list(df.columns)


['name',
 'description',
 'neighborhood_overview',
 'host_name',
 'host_location',
 'host_about',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_listings_count',
 'host_total_listings_count',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood_group_cleansed',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'has_availability',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'number_of_reviews',
 'number_of_reviews_ltm',
 'number_of_reviews_l30d',
 'review_scores_rating',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'instant_bookable',
 'calculated_host_listings_count',
 'cal

In [3]:

dropping_col = [ 'host_name',
 'host_location',
 'host_about',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_listings_count',
 'host_total_listings_count',
 'host_has_profile_pic',
 'host_identity_verified',
 'n_host_verifications',
'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms', 'neighborhood_overview', 'name', 'description'] 

# Avoid errors if columns are missing
df = df.drop(columns=dropping_col, errors='ignore') 

list(df.columns)


['neighbourhood_group_cleansed',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'has_availability',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'number_of_reviews',
 'number_of_reviews_ltm',
 'number_of_reviews_l30d',
 'review_scores_rating',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'instant_bookable',
 'reviews_per_month']

In [4]:
# Check missing values
missing_val = df.isnull().sum()
print("Missing Values Before Filling:\n", missing_val)



Missing Values Before Filling:
 neighbourhood_group_cleansed       0
room_type                          0
accommodates                       0
bathrooms                          0
bedrooms                        2918
beds                            1354
amenities                          0
price                              0
minimum_nights                     0
maximum_nights                     0
minimum_minimum_nights             0
maximum_minimum_nights             0
minimum_maximum_nights             0
maximum_maximum_nights             0
minimum_nights_avg_ntm             0
maximum_nights_avg_ntm             0
has_availability                   0
availability_30                    0
availability_60                    0
availability_90                    0
availability_365                   0
number_of_reviews                  0
number_of_reviews_ltm              0
number_of_reviews_l30d             0
review_scores_rating               0
review_scores_cleanliness          0
review

In [5]:

# Fill missing values: numeric columns -> mean, categorical columns -> 'Unknown'
for col in df.columns:
    if df[col].dtype == 'object':  # Categorical column
        df[col].fillna('Unknown', inplace=True)
    else:  # Numeric column
        df[col].fillna(df[col].mean(), inplace=True)


# Check missing values after filling
missing_val_after = df.isnull().sum()
print("Missing Values After Filling:\n", missing_val_after)

# Display updated column list
print("Updated Columns:\n", list(df.columns))

Missing Values After Filling:
 neighbourhood_group_cleansed    0
room_type                       0
accommodates                    0
bathrooms                       0
bedrooms                        0
beds                            0
amenities                       0
price                           0
minimum_nights                  0
maximum_nights                  0
minimum_minimum_nights          0
maximum_minimum_nights          0
minimum_maximum_nights          0
maximum_maximum_nights          0
minimum_nights_avg_ntm          0
maximum_nights_avg_ntm          0
has_availability                0
availability_30                 0
availability_60                 0
availability_90                 0
availability_365                0
number_of_reviews               0
number_of_reviews_ltm           0
number_of_reviews_l30d          0
review_scores_rating            0
review_scores_cleanliness       0
review_scores_checkin           0
review_scores_communication     0
review_scores_loc

/var/folders/nf/lqj62v5n3pj50v8m9psrn48c0000gp/T/ipykernel_34805/3499057956.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna('Unknown', inplace=True)
/var/folders/nf/lqj62v5n3pj50v8m9psrn48c0000gp/T/ipykernel_34805/3499057956.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave

In [6]:

# Separate categorical and numerical features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(exclude=['object']).columns

# Convert 'price' to numeric, removing non-numeric characters (like '$' or commas)
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)



In [7]:
# Convert 'amenities' column (which contains lists as strings) into actual lists
df['amenities'] = df['amenities'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Convert each list of amenities into a set
df['amenities'] = df['amenities'].apply(set)

In [8]:
# Gather all unique amenities
all_amenities = set()

for amenities in df['amenities']:
    all_amenities.update(amenities)


    


In [9]:

# Convert amenities into binary columns (1 if present, 0 if not)
amenities_df = pd.DataFrame([{amenity: (amenity in amenities) for amenity in all_amenities} for amenities in df['amenities']])

# Drop original 'amenities' column and merge the new binary features
df = df.drop(columns='amenities')
df = df.join(amenities_df)

# One-hot encode categorical features
categorical_features = ['neighbourhood_group_cleansed', 'room_type']
df = pd.get_dummies(df, columns=categorical_features)

# Display final dataset structure
print("Final DataFrame Shape:", df.shape)
print("Final Columns:", df.columns)




Final DataFrame Shape: (28022, 2813)
Final Columns: Index(['accommodates', 'bathrooms', 'bedrooms', 'beds', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       ...
       'GE stainless steel stove', 'neighbourhood_group_cleansed_Bronx',
       'neighbourhood_group_cleansed_Brooklyn',
       'neighbourhood_group_cleansed_Manhattan',
       'neighbourhood_group_cleansed_Queens',
       'neighbourhood_group_cleansed_Staten Island',
       'room_type_Entire home/apt', 'room_type_Hotel room',
       'room_type_Private room', 'room_type_Shared room'],
      dtype='object', length=2813)


In [10]:
df.head()

,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,...,GE stainless steel stove,neighbourhood_group_cleansed_Bronx,neighbourhood_group_cleansed_Brooklyn,neighbourhood_group_cleansed_Manhattan,neighbourhood_group_cleansed_Queens,neighbourhood_group_cleansed_Staten Island,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,1,1.0,1.329708,1.0,150.0,30,1125,30.0,30.0,1125.0,...,False,False,False,True,False,False,True,False,False,False
1,3,1.0,1.000000,3.0,75.0,1,730,1.0,1.0,730.0,...,False,False,True,False,False,False,True,False,False,False
2,4,1.5,2.000000,2.0,275.0,5,1125,5.0,5.0,1125.0,...,False,False,True,False,False,False,True,False,False,False
3,2,1.0,1.000000,1.0,68.0,2,14,2.0,2.0,14.0,...,False,False,False,True,False,False,False,False,True,False
4,1,1.0,1.000000,1.0,75.0,2,14,2.0,2.0,14.0,...,False,False,False,True,False,False,False,False,True,False


In [11]:
def labels(columns):
    for col in columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
    return df


categorical_features = df.select_dtypes(include=['bool']).columns
print("Boolean Columns:", print(len(categorical_features)), list(categorical_features))



df = labels(categorical_features)
df.head()

2786
Boolean Columns: None ['has_availability', 'instant_bookable', "Dr Bronner's Peppermint Soap body soap", 'Beatbox and JBL sound system with Bluetooth and aux', 'Irish Spring  body soap', 'Avanti refrigerator', '42" HDTV with Netflix, HBO Max, Apple TV, Amazon Prime Video', 'Fast wifi – 490 Mbps', '55" HDTV with Netflix, Amazon Prime Video, standard cable, Chromecast', 'Bath and bodywork  body soap', 'Irish Spring and Dove body soap', 'SONUS and Samsung Sound Bar with Bass sound system with Bluetooth and aux', '40" HDTV with Netflix, Roku', 'Extra pillows and blankets', 'GE stainless steel gas stove', 'Frigidaire gas stove', 'Local shampoo', 'Fast wifi – 425 Mbps', 'Fast wifi – 160 Mbps', "Trader Joe's body soap", 'Everybody body soap', 'Pantene, Head & Shoulders conditioner', 'Organic  conditioner', 'Children’s books and toys for ages 2-5 years old', 'Samsung oven', 'MIele  stainless steel gas stove', 'Kirkland shampoo', 'Fast wifi – 83 Mbps', 'palmolive conditioner', 'HDTV with A

,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,...,GE stainless steel stove,neighbourhood_group_cleansed_Bronx,neighbourhood_group_cleansed_Brooklyn,neighbourhood_group_cleansed_Manhattan,neighbourhood_group_cleansed_Queens,neighbourhood_group_cleansed_Staten Island,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,1,1.0,1.329708,1.0,150.0,30,1125,30.0,30.0,1125.0,...,0,0,0,1,0,0,1,0,0,0
1,3,1.0,1.000000,3.0,75.0,1,730,1.0,1.0,730.0,...,0,0,1,0,0,0,1,0,0,0
2,4,1.5,2.000000,2.0,275.0,5,1125,5.0,5.0,1125.0,...,0,0,1,0,0,0,1,0,0,0
3,2,1.0,1.000000,1.0,68.0,2,14,2.0,2.0,14.0,...,0,0,0,1,0,0,0,0,1,0
4,1,1.0,1.000000,1.0,75.0,2,14,2.0,2.0,14.0,...,0,0,0,1,0,0,0,0,1,0


In [14]:
X = df.drop(columns="price")
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print(X_train.shape, X_test.shape, y_train.shape, y_test)

(19615, 2812) (8407, 2812) (19615,) 10588     95.0
4146      89.0
5106     119.0
3236     196.0
4770     175.0
         ...  
5320      50.0
14010     87.0
19033     40.0
23093    425.0
25949     89.0
Name: price, Length: 8407, dtype: float64


In [16]:
params = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10],
}

dt = DecisionTreeRegressor()
grid_search = GridSearchCV(dt, params, cv=5, scoring="accuracy")
grid_search.fit(X_train, y_train)

dt = grid_search.best_estimator_
predictions = dt.predict(X_test)

print(predictions)

/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/utils/_param_validation.py", line 213,

[ 48. 250. 105. ...  55. 435. 115.]


In [20]:
accuracy = accuracy_score(y_train, dt.predict(X_train))
print(accuracy)

0.999898037216416
